# the start of TensorFlow2 with MNIST

## Import TensorFlow into your program:

In [3]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

print("The version of TensorFlow in this source code: {}".format(tf.__version__))

The version of TensorFlow in this source code: 2.4.1


## Load and prepare the MNIST dataset.

In [4]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# Add a channels dimension
x_train = x_train[..., tf.newaxis].astype("float32")
x_test = x_test[..., tf.newaxis].astype("float32")

## Use tf.data to batch and shuffle the dataset:

In [5]:
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32)

test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

## Build the tf.keras model using the Keras model subclassing API:

In [6]:
class MyModel(Model):
  def __init__(self):
    super(MyModel, self).__init__()
    self.conv1 = Conv2D(32, 3, activation='relu')
    self.flatten = Flatten()
    self.d1 = Dense(128, activation='relu')
    self.d2 = Dense(10)

  def call(self, x):
    x = self.conv1(x)
    x = self.flatten(x)
    x = self.d1(x)
    return self.d2(x)

# Create an instance of the model
model = MyModel()

## Choose an optimizer and loss function for training:

In [7]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

optimizer = tf.keras.optimizers.Adam()

## Select metrics to measure the loss and the accuracy of the model. These metrics accumulate the values over epochs and then print the overall result.

In [8]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

## Use tf.GradientTape to train the model:

In [9]:
@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape:
    # training=True is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    predictions = model(images, training=True)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)

## Test the model:

In [10]:
@tf.function
def test_step(images, labels):
  # training=False is only needed if there are layers with different
  # behavior during training versus inference (e.g. Dropout).
  predictions = model(images, training=False)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)

## Set epochs and training

In [11]:
EPOCHS = 5

for epoch in range(EPOCHS):
  # Reset the metrics at the start of the next epoch
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()

  for images, labels in train_ds:
    train_step(images, labels)

  for test_images, test_labels in test_ds:
    test_step(test_images, test_labels)

  print(
    f'Epoch {epoch + 1}, '
    f'Loss: {train_loss.result()}, '
    f'Accuracy: {train_accuracy.result() * 100}, '
    f'Test Loss: {test_loss.result()}, '
    f'Test Accuracy: {test_accuracy.result() * 100}'
  )

Epoch 1, Loss: 0.13907039165496826, Accuracy: 95.836669921875, Test Loss: 0.05742891505360603, Test Accuracy: 97.98999786376953
Epoch 2, Loss: 0.044048912823200226, Accuracy: 98.63166809082031, Test Loss: 0.05825277417898178, Test Accuracy: 98.1199951171875
Epoch 3, Loss: 0.023009363561868668, Accuracy: 99.26333618164062, Test Loss: 0.05366259440779686, Test Accuracy: 98.41999816894531
Epoch 4, Loss: 0.013780804350972176, Accuracy: 99.57333374023438, Test Loss: 0.060097746551036835, Test Accuracy: 98.22999572753906
Epoch 5, Loss: 0.009887662716209888, Accuracy: 99.65833282470703, Test Loss: 0.06167370453476906, Test Accuracy: 98.32999420166016


# reference

### https://tensorflow.google.cn/tutorials/quickstart/advanced